#### Шаг 0. Импорт и получение токена

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import asyncio
import nest_asyncio; nest_asyncio.apply()
from datetime import datetime as dt
import requests

#увеличиваем количество столбцов и строк для просмотра
pd.set_option('display.max_rows', 200,'display.max_columns', 200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# from rudata.RuDataDF import RuDataDF, RuDataRequest
# RuDataRequest.set_headers()

In [ ]:
#Датой должен быть рабочий день
file = 'ISIN_01.2023.xlsx'
date1 = '2024-03-31'
# date1 = dt.today().strftime("%Y-%m-%d")

In [3]:
cashFlowCalcDate = date1
prices1, prices0 = pd.DataFrame(), pd.DataFrame()

In [39]:
req_date = dt.strptime(date1, "%Y-%m-%d").strftime("%d.%m.%Y")

# файл fond_market_rates
url = f"https://www.nationalclearingcentre.ru/api/v1/rates/fondMarketRates?action=xls&lang=ru&pageNumber=1&type=1&secId=&date={req_date}&date1={req_date}&date2={req_date}"
fond_market_rates = requests.get(url, headers={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"})
with open(f"data/Input/fond_market_rates_{dt.strptime(date, '%Y-%m-%d').strftime('%Y%m%d')}.xlsx", "wb") as fond_market_rates_writer:
    fond_market_rates_writer.write(fond_market_rates.content)
    
# файл secur_info
url = f"https://www.nationalclearingcentre.ru/api/v1/rates/securInfo?action=xls&lang=ru&pageNumber=1&&settleDate={req_date}&secur=&isin=&fullCovered=&fullCoveredLimit=&otcDeals=&collateral=&gfUse=&sfUse=&gcIRisks=&gcCollateral=&gcBonds=&gcShares=&gcExpanded=&gcOfz=&gcMetal=&maturityDate=&collateralAcceptLimit="
secur_info = requests.get(url, headers={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"})
with open(f"data/Input/secur_info_{dt.strptime(date1, '%Y-%m-%d').strftime('%Y%m%d')}.xlsx", "wb") as secur_info_writer:
    secur_info_writer.write(secur_info.content)

#### Шаг 1. Загрузка stage таблиц 

Rudata

#### Loading RuDataDataFrames

[FintoolReferenceData](https://docs.efir-net.ru/dh2/#/Info/FintoolReferenceData?id=post-fintoolreferencedata) - Получить расширенный справочник по финансовым инструментам.

In [4]:
FintoolReferenceData = RuDataDF("FintoolReferenceData").df
FintoolReferenceData.head()

,facevalue_m,faceftname_m,sumissuevol_m,summarketvol_m,numcoupons_m,nrdcode,regcode_m,regdate_m,regdistdate_m,begdistdate_m,enddistdate_m,endmtydate_m,daysall_m,fullname_nrd,rpregorg_nrd,rpstate_nrd,issuenumber,currentfacevalue_nrd,isqualified_nrd,bondseries,islombardcbr_nrd,numcoupons_nrd,facevalue_nrd,faceftname_nrd,summarketvol_nrd,regcode_nrd,regdate_nrd,regorg_nrd,begdistdate_nrd,daysall_nrd,enddistdate_nrd,endmtydate_nrd,regdistdate_nrd,sumissuevol_nrd,drqty,isincodebase_nrd,nrdcompcode,shqty,summarketvolorig_nrd,issuername_nrd,planmtydate_nrd,firstcoupondate_nrd,depositary_nrd,registrar_nrd,placeprice_nrd,micexlist_nrd,minlot_nrd,formarketbonds_nrd,registraracctype_nrd,registraracctypedate_nrd,facialacc_nrd,acc_open_date_nrd,fraction_nrd,specialized_depo_nrd,cmp_code_nsd_issuer_nrd,cmp_code_nsd_managing_nrd,ratetypenamerus_nrd,ratetypenameeng_nrd,coupontypename_nrd,base_month,base_year,basis_nrd,sectypenamerus_nrd,sectypenameeng_nrd,secformnamerus_nrd,secformnameeng_nrd,sectypenamebr_nrd,secstaterus_nrd,secstateeng_nrd,placementtype_nrd,share_category_name_nrd,share_category_name_en_nrd,dr_category_name_nrd,dr_category_name_en_nrd,company_state_name_nrd,rate_type_id,cp_type_id,coupon_period_base_id,sec_type_id,sec_form_id,sec_state_id,placement_type_id,share_category_id,dr_category_mn,company_state_mn,sector4212u,sna2008,fullname_en_nrd,seniorityname,fintoolid,moex_code,moex_name,fullname,isin144a,isincode,nickname,progfintoolid,regcode,regdate,regorg,amortisedmty,basis,bondstructuralpar,cfi,coupontype,fintooltype,guaranteeamount,guaranteetype,haveoffer,isconvertible,isguaranteed,maturitygroup,securitization,accruedintcalctype,country,couponperyear,faceftname,facevalue,guarantval,havedefault,numcoupons,status,sumissueval,sumissuevol,summarketval,summarketvol,borrowername,borrowerokpo,borrowersector,borroweruid,borrowerinn,borrowercountry,borrowerregionname,issuername,issuerokpo,issuersector,issueruid,issuerinn,issuercountry,issuerregionname,assetssector,numguarantors,begdistdate,begmtydate,daysall,enddistdate,endmtydate,privatedist,regdistdate,securitytype,securitykind,nominaltype,update_time,isregion,nsd_service,raterevisionsperyear,floatratename,issubordinated,note,listing_level,havecovenant,haverepayment,haveindexedfv,numdecimals,couponperiod,ismatched,firstcoupondate,couponrate,iscomplicated,available_for_unqualified,sec_type_br_code,original_country,qualified_test_id,qualified_test_id_tm,base_asset_fintoolid,isqualified,listing_level_spb,have_mm_moex,have_mm_spb,beg_book_date,seniority,tradesite,counter,rn
0,2.0,2.0,2.0,2.0,2.0,None,2.0,0.0,2.0,2.0,2.0,2.0,2.0,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,S111,None,Старшие необеспеченные,4000,None,None,"МинФин РФ, государственные краткосрочные беску...",None,None,ГКО-21134,NaN,21134RMFS,None,МинФин,0.0,act/365,None,None,Дисконтный,Облигация,None,Необеспеченные,0.0,0.0,0.0,Краткосрочные,None,1.0,RU,0.0,RUB,1000.0,NaN,0.0,0.0,Погашен,2.500000e+09,2500000.0,2.500000e+09,2500000.0,Минфин РФ,00013474,Государственные,6874.0,7710168360,RU,г.Москва,Минфин РФ,00013474,Государственные,6874.0,7710168360,RU,г.Москва,None,0.0,1999-12-17T00:00:00,None,110.0,1999-12-17T00:00:00,2000-04-05T00:00:00,0.0,1999-12-17T00:00:00,Гос,Классические,Постоянный,2023-04-19T15:55:00,NaN,Нет информации,0.0,None,NaN,None,NaN,False,False,False,2.0,NaN,True,None,0.0,False,False,None,RU,1000,1000,NaN,False,NaN,False,False,None,416.0,NaN,97052,1
1,3.0,3.0,3.0,4.0,3.0,SU21135RMFS7,3.0,1.0,2.0,3.0,2.0,3.0,2.0,Государственные краткосрочные бескупонные обли...,Финансовые органы,None,135,0.0,0.0,None,0.0,0.0,1000.0,RUB,1500000.0,21135RMFS,1999-12-10T00:00:00,Финансовые органы,1999-12-21T00:00:00,NaN,None,2000-05-17

In [5]:
fintoolIds = list(set(FintoolReferenceData['fintoolid']))

In [6]:
cashFlowCalcDate = "2024-02-29"
AccruedInterestOnDateBefore = RuDataDF("AccruedInterestOnDate").set_date(cashFlowCalcDate).df
AccruedInterestOnDate.head()

,fintoolId,shiftDays,shiftTypeSource,typeCurrency,date,accruedInterest,accruedInterestPct,currentFaceValue,currentFaceValuePct,calculationType,error,faceValueCurrency,accruedInterestCurrency,faceValueLastKnownDate
0,4097,0,Calendar,None,2024-02-29T00:00:00,0.55,0.054781,1000.0,100.0,Typical,None,RUB,RUB,2024-02-29T00:00:00
1,4096,0,Calendar,None,2024-02-29T00:00:00,10.57,1.057281,1000.0,100.0,Typical,None,RUB,RUB,2024-02-29T00:00:00
2,4091,0,Calendar,None,2024-02-29T00:00:00,NaN,NaN,1000.0,100.0,Typical,None,RUB,RUB,2024-02-29T00:00:00
3,4090,0,Calendar,None,2024-02-29T00:00:00,NaN,NaN,1000.0,100.0,Typical,None,RUB,RUB,2024-02-29T00:00:00
4,4083,0,Calendar,None,2024-02-29T00:00:00,NaN,NaN,1000.0,100.0,Typical,None,RUB,RUB,2024-02-29T00:00:00


In [ ]:
cashFlowCalcDate = "2024-03-29"
AccruedInterestOnDateAfter = RuDataDF("AccruedInterestOnDate").set_date(cashFlowCalcDate).df
AccruedInterestOnDate.head()

In [9]:
ExchangeTree = pd.read_excel("data/dictionaries/ExchangeTree.xlsx")
ExchangeTree.head()

,Unnamed: 0,id,id_parent,shortname_rus,shortname_eng,fullname_rus,fullname_eng,sort_order,update_date,counter,rn
0,0,144,63,Нал,Cash,Наличная валюта,Cash,0,2020-12-28T17:03:09,237,1
1,1,270,63,RUDIP 2,RUDIP 2,RUDIP 2,RUDIP 2,1,2022-10-31T19:41:24,237,2
2,2,84,83,США,US,Макроиндикаторы США,USA macroindicators,1,2020-12-28T17:03:09,237,3
3,3,85,83,РФ,RU,Макроиндикаторы Россия,Russia macroindicators,2,2020-12-28T17:03:09,237,4
4,4,2,1,РТС,RTS,РТС индексы,RTS indexes,2,2020-12-28T17:03:09,237,5


In [10]:
InfoSecurities = RuDataDF("InfoSecurities").df
InfoSecurities.head()

,id_fintool,quatro_id,shortname_rus,fullname_rus,isin,cfi,reg_code,on_quatro,facevalue,facevalue_currency,issue_vol,coupon_rate,endmty_date,id_emitent,fintoolid,fininstid,emitent_shortname_rus,okpo,inn,ogrn,sector,update_date,fintooltype,securitytype,securitykind,status_name,coupons_per_year,borroweruid,borrowername,borrowerinn,borrowersector,nsd_service,nrdcode,id_iss,listing_level,reg_date,counter
0,1,None,Amur Minerals-ао,"Amur Minerals Corporation, акции обыкновенные",VGG042401007,ESVUFR,None,1,NaN,GBP,NaN,None,None,33692.0,22907,93313.0,Amur Minerals Corp,None,None,None,Горнодобыча,2024-02-03T06:24:22,Акция,Обыкн-ая,Осн-ой,В обращении,NaN,NaN,None,None,None,На обслуживании,VGG042401007,922622.0,NaN,None,97054.0
1,2,None,Amtel Vredestein GDR,Глобальные депозитарные расписки на обыкновенн...,US03235R1014,None,None,1,1.00,USD,NaN,None,None,30738.0,17832,87579.0,Bank of New York Mellon,None,135160382,None,Банки,2024-02-19T13:20:42,Депозитарная расписка,GDR,Неопределено,Погашен,NaN,88174.0,Amtel,81096744301,Автомобили и запчасти,Нет информации,None,NaN,NaN,None,97054.0
2,3,None,CTC Media-ао,"CTC Media, акции обыкновенные",US12642X1063,ESVUFR,None,1,0.01,USD,157300070.0,None,2016-05-20T00:00:00,29653.0,20584,89961.0,CTC Media,None,581869211,None,СМИ,2023-10-26T13:36:35,Акция,Обыкн-ая,Осн-ой,Погашен,NaN,NaN,None,None,None,На обслуживании,US12642X1063,852752.0,NaN,None,97054.0
3,5,None,Etalon Group GDR RegS,Глобальные депозитарные расписки на акции Etal...,US29760G1031,EDSXFR,None,1,1.00,USD,NaN,None,None,29662.0,64928,126310.0,RCS Issuer Services,None,None,None,Финанс.сервисы,2024-02-27T06:45:02,Депозитарная расписка,GDR,Regulation S,В обращении,NaN,94104.0,Etalon Group,None,Строительство,На обслуживании,US29760G1031,2928807.0,NaN,None,97054.0
4,6,None,Evraz-ао,"Evraz plc, акции обыкновенные",GB00B71N6K86,ESVUFR,None,1,0.05,USD,NaN,None,None,29693.0,72782,94280.0,Evraz,None,None,None,Металлургический,2024-02-03T06:24:22,Акция,Обыкн-ая,Осн-ой,В обращении,NaN,NaN,None,None,None,На обслуживании,GB00B71N6K86,3119519.0,NaN,None,97054.0


In [11]:
MoexSecurities = RuDataDF("MoexSecurities").df
MoexSecurities.head()

,id_iss,secid,boardid,board_title,board_group,board_group_title,market,engine,short_name,isin,cfi_ifx,cfi_name,lot_size,expiration_date,currency,emitent_code,emitent_name,market_sector,min_price_increment,fintoolid,reg_code,update_date,counter,rn
0,2501,EUR_RUB__TOM,CETS,Системные сделки - безадрес.,currency,Системные сделки - безадрес.,selt,currency,Евро/Рубль-Том,None,YCRSXX,Валютные пары / Тип пары / Кросс-курс,1000,None,RUB,None,None,None,0.0025,NaN,None,2021-02-28T23:56:16.686054,179369,1
1,2502,EUR_RUB__TOD,CETS,Системные сделки - безадрес.,currency,Системные сделки - безадрес.,selt,currency,Евро/Рубль-Тод,None,YCRSXX,Валютные пары / Тип пары / Кросс-курс,1000,None,RUB,None,None,None,0.0025,NaN,None,2021-02-28T23:56:16.686053,179369,2
2,2840,KZT000000TOM,CETS,Системные сделки - безадрес.,currency,Системные сделки - безадрес.,selt,currency,Тенге/Рубль-Тод,None,YCRXXX,Валютные пары / Тип пары / Кросс-курс,10000,None,RUB,None,None,None,0.0025,NaN,None,2023-03-23T18:16:31.385678,179369,3
3,3691,UAH000000TOM,CETS,Системные сделки - безадрес.,currency,Системные сделки - безадрес.,selt,currency,Гривна/Рубль-Тод,None,YCRSXX,Валютные пары / Тип пары / Кросс-курс,1000,None,RUB,None,None,None,0.0025,NaN,None,2018-09-17T09:17:51.410715,179369,4
4,3752,USD000000TOD,CETS,Системные сделки - безадрес.,currency,Системные сделки - безадрес.,selt,currency,Доллар/Рубль-Тод,None,YCRSXX,Валютные пары / Тип пары / Кросс-курс,1000,None,RUB,None,None,None,0.0025,NaN,None,2021-02-28T23:56:17.686097,179369,5


In [5]:
RUPriceHistory = RuDataDF("RUPriceHistory").df
RUPriceHistory.shape

RUPriceHistory started
RUPriceHistory finished. RUPriceHistory shape (38990, 82)


(38990, 82)

In [16]:
Calendar = RuDataDF("Calendar").df

ClientConnectorError: Cannot connect to host dh2.efir-net.ru:443 ssl:default [Удаленный хост принудительно разорвал существующее подключение]

In [ ]:
Calendar.to_parquet("Calendar.parquet")

In [ ]:
'https://dh2.efir-net.ru/v2/Archive/CurrencyRate'

In [ ]:
'https://dh2.efir-net.ru/v2/Archive/EndOfDay'

#### Tranforming datagrames and loading to excel

In [ ]:
NSDFintoolReferenceData = FintoolReferenceData.copy()[['isincode', 'nickname', 'fintooltype', 'faceftname']]
# FintoolReferenceData = FintoolReferenceData.loc[FintoolReferenceData.isincode.isin(empty_isins)]
NSDFintoolReferenceData = NSDFintoolReferenceData.drop_duplicates(subset=['isincode'])
NSDFintoolReferenceData.columns = ['ISIN', 'Name', 'Type', 'Currency']
NSDFintoolReferenceData['ISIN in DB'] = True
NSDFintoolReferenceData = NSD.set_index('ISIN', drop=True)

In [ ]:
TransformedInfoSecurities = InfoSecurities.copy()[['isin', 'fintoolid']].drop_duplicates(subset=['isin'])
TransformedInfoSecurities.columns = ['ISIN', 'Fintool']
TransformedInfoSecurities = TransformedInfoSecurities.set_index('Fintool', drop=True)
fintools = TransformedInfoSecurities['ISIN'].to_dict()

In [ ]:
NSDAccruedInterestOnDate = AccruedInterestOnDate.copy()
NSDAccruedInterestOnDate = NSDAccruedInterestOnDate[(~NSDAccruedInterestOnDate['accruedInterest'].isna()) & (~NSDAccruedInterestOnDate['currentFaceValue'].isna())]
NSDAccruedInterestOnDate['ISIN'] = NSDAccruedInterestOnDate['fintoolId'].apply(lambda x: fintools[x])
NSDAccruedInterestOnDate = NSDAccruedInterestOnDate[['ISIN','accruedInterest','currentFaceValue','faceValueCurrency']].set_index('ISIN')
NSDAccruedInterestOnDate.columns = ['Interest', 'Principal', 'Currency']
NSDAccruedInterestOnDate['ISIN in DB'] = True

In [ ]:
'https://dh2.efir-net.ru/v2/RUPrice/History'

In [ ]:
ExchangesFintoolReferenceData = FintoolReferenceData.copy()[['isincode', 'nickname', 'fintooltype', 'faceftname']].drop_duplicates(subset=['isincode'])
ExchangesFintoolReferenceData.columns = ['ISIN', 'Name', 'Type', 'Currency']
ExchangesFintoolReferenceData['ISIN in DB'] = True
ExchangesFintoolReferenceData =ExchangesFintoolReferenceData.set_index('ISIN', drop=True)

In [ ]:
#Датой должен быть рабочий день
file = 'ISIN_01.2023.xlsx'
date0 = '2022-02-18'
date1 = '2023-01-31'
C1 = p.Current_prices(file, date1)
C1_prices = C1.get_prices()


In [ ]:
C1.prices.loc[C1.prices['ISIN in DB']==False]

In [ ]:
common_data.update(TransformedFintoolReferenceData)

In [ ]:
C0 = p.Current_prices(file, date0, adjusting_date=date1, show_output=True other_date_df=C1_prices)
C0.prices = C0.prices.loc[C1.prices.index:]
C0_prices = C0.get_prices()

In [ ]:
#расчет множителя корректировки цены в отчетности
#формула множителя будет для облигаций RUBprice1/((P%0*N1+ NKD1)*FX0), для акций RUBprice1/RUBprice0
prices1 = pd.merge(C1.prices, C0.prices[['Clean price pct','FX Rate','RUB Price']], left_index=True, right_index=True, how="left",suffixes=['','_0'])
prices1[['Price change']] = np.nan
prices1.loc[prices1['Type']=='Облигация','Price change'] = \
prices1['RUB Price']/((prices1['Clean price pct_0']*prices1['Principal']/100+ prices1['Interest'])*prices1['FX Rate_0'])
prices1.loc[prices1['Type']!='Облигация', 'Price change']= \
prices1['RUB Price']/prices1['RUB Price_0']
prices1 = prices1.drop(columns=['Clean price pct_0', 'FX Rate_0', 'RUB Price_0'])

In [ ]:
prices1

In [ ]:
with pd.ExcelWriter(f'Output/prices_{C1.date.replace("-","")}_with_change.xlsx',datetime_format='dd.mm.yyyy') as writer:  
    prices1.to_excel(writer)

In [ ]:
obj = yf.Ticker('AMZN')
info = obj.get_info(timeout=10)
start_date = (dt.strptime(C0.date,"%Y-%m-%d")-relativedelta(months=1)).strftime("%Y-%m-%d")
df = obj.history('AMZN',
               start=start_date,
               auto_adjust=False,
               debug=True,
               timeout=10)
if df.loc[:C0.date,:].shape[0]==0:
    pass
#Un-correcting for dividends and splits
df['Div mult'] = 1-df['Dividends']/df['Close'].shift(1)
df['Div mult'] = df['Div mult'].prod()/df['Div mult'].cumprod()*df['Div mult']
df['Div mult'] = df['Div mult'].shift(-1)
df.loc[df.index[-1],'Div mult'] = 1
df.loc[df['Stock Splits']==0,'Stock Splits'] = 1
df['Split mult'] = 1/df['Stock Splits']
df['Split mult'] = df['Split mult'].prod()/df['Split mult'].cumprod()*df['Split mult']
df['Split mult'] = df['Split mult'].shift(-1)
df.loc[df.index[-1],'Split mult'] = 1
df['Mult'] = df['Split mult']*df['Div mult'] 
df['Real Close'] = df['Adj Close']/df['Mult']